In [53]:
import os
import torch
import transformers
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [54]:
def load_documents():
    docs = []
    folder_path = '../data/docs'
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as f:
                text = f.read()
                docs.append(Document(page_content=text, metadata={'source': filename}))
    return docs
docs = load_documents()
print(docs)

[Document(metadata={'source': 'HistoricalTrends.txt'}, page_content='Historical Trends and Known Patterns in Financial Markets\nUnderstanding historical trends and patterns in financial markets is key for investors to make informed decisions. Financial markets are influenced by numerous factors—economic conditions, geopolitical events, technological advancements, and investor sentiment, to name a few. Over the years, patterns have emerged, and while markets are unpredictable in the short term, they tend to follow certain trends over time.\n\nThis document will delve deeply into the most significant historical trends and patterns in stock, bond, commodity, and currency markets, illustrating them with real-world examples. We’ll explore common market cycles, behavioral patterns like bubbles and crashes, and factors such as interest rates, inflation, and geopolitical events that influence market dynamics.\n\n1. Stock Market Trends and Patterns\n1.1. Market Cycles\nMarket cycles describe th

In [55]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs)

In [5]:
os.environ["TAVILY_API_KEY"] = "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"

In [8]:
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = TavilySearchResults(k=3).invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

In [13]:
state = web_search({"question": "How is Apple doing? Should I invest", "documents": None})

---WEB SEARCH---


In [14]:
print(state["question"])
for doc in state["documents"]:
    print(doc)

How is Apple doing? Should I invest
page_content='In the past 20 years, Apple has been one of the best investments in the entire market, generating a total return of roughly 54,730%. However, Apple is now the most valuable company in the U.S ...
Net sales also increased by 8%, driven by the launch of new products and services like the updated 14-inch MacBook Pro, the third-generation version of the popular AirPods, the launch of the iPhone 14 and the introduction of the Apple Watch Series 8. But in 2022, its stock price dropped from a high of $182.88 on the year’s first trading day, Jan. 3, to close at $129.93 by the end of December—a decrease of nearly 29%—as the company continued to battle production and supply issues.
 But if you had invested $500 in the S&P 500 index in 2018, you’d have $738.
Opting to invest in Apple would allow your money to grow more than twice as much than if you had invested in the S&P 500.
 Apple Stock Forecast
Reviewed By
Reviewed By
Published: Mar 22, 2023,

In [2]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

KeyboardInterrupt: 

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])